## Example: Converting a Segmentation Stack for Use with TrackGardener

This notebook provides a practical example of converting a segmentation stack - where each object’s value corresponds to its unique TrackID - to a TrackGardener database using the configuration file `Fluo-N2DL-HeLa-01_config_trackID.yaml`.

In this workflow, relationships between parent and offspring tracks are inferred based on their proximity in time and space. This approach is generally error-prone and not recommended for most applications. For more accurate lineage relationships, use an explicit graph structure; see labels_and_geff_to_TrackGardener for details of importing the graph structure from the `geff` format.

You can reuse the configuration file demonstrated here with the TrackGardener plugin itself. Just be sure to specify the correct paths to your imaging dataset and the TrackGardener database within the config file. Using absolute (rather than relative as for the purpose of this example) paths is the safest choice to ensure your files are correctly located.

In [1]:
import dask.array as da
import yaml

from track_gardener.converters.track_array_2_gardener import (
    assign_tracks_relationships,
    convert_array_segmentations_to_db,
)

In [2]:
# pathways to the segmentation stack and configuration file

segm_path = './Fluo-N2DL-HeLa-01_segm.zarr/labels'
config_path = './Fluo-N2DL-HeLa-01_config_trackID.yaml'

In [3]:
# read segmentation array
segm_array = da.from_zarr(segm_path)

In [4]:
# read in configuration file
with open(config_path) as file:
    config = yaml.safe_load(file)

In [5]:
# convert and save to Track Gardener database
convert_array_segmentations_to_db(segm_array, config)

2025-07-28 09:10:04.485 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=0
2025-07-28 09:10:04.533 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=1
2025-07-28 09:10:04.578 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=2
2025-07-28 09:10:04.622 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=3
2025-07-28 09:10:04.688 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=4
2025-07-28 09:10:04.767 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=5
2025-07-28 09:10:04.847 | INFO     | track_gardener.converters.track_array_2_garde

In [7]:
# guess parent-offspring relationships by space-time proximity
db_path = config['database']['path']
assign_tracks_relationships(db_path=db_path, parent_radius=100)

### Validate the resulting database

In [13]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from track_gardener.db.db_validate import run_tracking_db_checks

In [10]:
engine = create_engine(f'sqlite:///{db_path}')
session = sessionmaker(bind=engine)()

In [14]:
run_tracking_db_checks(session)

['Component {204, 207, 208, 211, 212, 220, 221} has multiple root values: {204, 207, 208, 211, 212, 220, 221}',
 'Component {225, 227, 228, 230, 231, 233, 234} has multiple root values: {225, 227, 228, 230, 231, 233, 234}',
 'Track 207 has root 207 but parent 204 has root 204',
 'Track 208 has root 208 but parent 207 has root 207',
 'Track 208 has root 208 but parent 204 has root 204',
 'Track 211 has root 211 but parent 220 has root 220',
 'Track 211 has root 211 but parent 204 has root 204',
 'Track 220 has root 220 but parent 204 has root 204',
 'Track 212 has root 212 but parent 220 has root 220',
 'Track 212 has root 212 but parent 204 has root 204',
 'Track 221 has root 221 but parent 207 has root 207',
 'Track 221 has root 221 but parent 204 has root 204',
 'Track 227 has root 227 but parent 225 has root 225',
 'Track 228 has root 228 but parent 225 has root 225',
 'Track 230 has root 230 but parent 228 has root 228',
 'Track 230 has root 230 but parent 225 has root 225',
 'Trac